In [16]:
"""
Following this kaggle
https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras/data?select=Sentiment.csv
"""

'\nFollowing this kaggle\nhttps://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras/data?select=Sentiment.csv\n'

In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [18]:
# import sys
#!{sys.executable} -m pip install keras

In [19]:
data = pd.read_csv('./kaggle/Sentiment.csv')

data = data[['text','sentiment']]
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print("Number of Positive Tweets: ", data[ data['sentiment'] == 'Positive'].size)
print("Number of Negative Tweets: ", data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [20]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
# Make Train and Test datasets
Y = pd.get_dummies(data['sentiment']).values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [22]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
225/225 - 29s - loss: 0.4346 - accuracy: 0.8147
Epoch 2/10
225/225 - 27s - loss: 0.3189 - accuracy: 0.8667
Epoch 3/10
225/225 - 27s - loss: 0.2764 - accuracy: 0.8840
Epoch 4/10
225/225 - 33s - loss: 0.2471 - accuracy: 0.8962
Epoch 5/10
225/225 - 28s - loss: 0.2228 - accuracy: 0.9094
Epoch 6/10
225/225 - 27s - loss: 0.2005 - accuracy: 0.9203
Epoch 7/10
225/225 - 27s - loss: 0.1810 - accuracy: 0.9247
Epoch 8/10
225/225 - 27s - loss: 0.1589 - accuracy: 0.9371
Epoch 9/10
225/225 - 28s - loss: 0.1474 - accuracy: 0.9400
Epoch 10/10
225/225 - 26s - loss: 0.1408 - accuracy: 0.9417


In [23]:
# save model 
model.save('./output/models/LSTM_unbalanced')

# to load and check model:

# load_model = tf.keras.models.load_model('./output/models/LSTM_unbalanced')
# load_model.summary()

INFO:tensorflow:Assets written to: ./output/models/LSTM_unbalanced\assets


In [56]:
y_pred = model.predict(X_test)
print('roc_auc_score', roc_auc_score(Y_test, y_pred))
y_pred=np.argmax(y_pred, axis=1) 
print('balanced accuracy score', balanced_accuracy_score(Y_test, y_pred))

roc_auc_score 0.8303390493914687


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [47]:
# roc_auc_score for minority (positive), and majority (negative) classes 
# get rows of positive and negative tweets
X_test_0 = X_test[Y_test[:,0]==1]
X_test_1 = X_test[Y_test[:,1]==1]
Y_test_0 = Y_test[Y_test[:,0]==1]
Y_test_1 = Y_test[Y_test[:,1]==1]

y_pred_0 = model.predict(X_test_0)
#y_pred_0 = np.argmax(y_pred_0, axis = 0)

y_pred_1 = model.predict(X_test_1)
#y_pred_1 = np.argmax(y_pred_1, axis = 1)

In [48]:
print(Y_test_0.shape)
print(Y_test_1.shape)
print(y_pred_0.shape)
print(y_pred_1.shape)
print(y_pred_0)

(2821, 2)
(720, 2)
(2821, 2)
(720, 2)
[[4.7007732e-02 9.5299232e-01]
 [9.9898726e-01 1.0127067e-03]
 [9.9981993e-01 1.8005547e-04]
 ...
 [9.9750143e-01 2.4985145e-03]
 [9.9995255e-01 4.7468777e-05]
 [9.9996948e-01 3.0460893e-05]]


In [49]:
print('roc_auc_score for negative tweets', accuracy_score(Y_test_0, y_pred_0))
print('roc_auc_score for positive tweets', accuracy_score(Y_test_1, y_pred_1))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets